In [1]:
#import pyodbc 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import sqlalchemy as sa
from sqlalchemy import create_engine, event
from sqlalchemy.engine.url import URL
import time
from  scipy.stats import zscore
import pyodbc 

### 1.. Reading input data

In [2]:
## Reading input data:
print ("connect to engine......")
#engine = sa.create_engine('mssql+pyodbc://' + "GREENMONKEY" + '/' + "Climate_Impact" + '?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')
engine = sa.create_engine('mssql+pyodbc://GREENMONKEY/Climate_Impact?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')

print ("connection to greenmonkey found")
print ("Reading wilfdire table from greenmonkey")
query=('''
SELECT *
  FROM [Climate_Impact].[wild_fire].[fireco2_10km_cube_trends]

''')

fireco2 = pd.read_sql(query, engine)
print(fireco2.head())

connect to engine......
connection to greenmonkey found
Reading wilfdire table from greenmonkey
         GridNum10km                                        LEVEL3_name  \
0  14434530383364096  Lochaber, Skye & Lochalsh, Arran & Cumbrae and...   
1  14434530383364096  Lochaber, Skye & Lochalsh, Arran & Cumbrae and...   
2  14434530383364096  Lochaber, Skye & Lochalsh, Arran & Cumbrae and...   
3  14435007124733952                       Perth & Kinross and Stirling   
4  14451091777257472                        East Lothian and Midlothian   

  LEVEL3_code  ADM_ID ISO2 protected_area env_zones LULUCF_CODE  \
0       UKM63  1352.0   GB           None       ATN          FL   
1       UKM63  1352.0   GB           None       ATN          GL   
2       UKM63  1352.0   GB      Protected       ATN          FL   
3       UKM77  1360.0   GB      Protected       ATN          GL   
4       UKM73  1357.0   GB           None       ATN          GL   

  LULUCF_DESCRIPTION  fire_2000  ...  burntCO2_ano

### 2..  Unpivot data

#### 2.1 Fire

In [3]:
# list columns in dataframe
list(fireco2.columns)

['GridNum10km',
 'LEVEL3_name',
 'LEVEL3_code',
 'ADM_ID',
 'ISO2',
 'protected_area',
 'env_zones',
 'LULUCF_CODE',
 'LULUCF_DESCRIPTION',
 'fire_2000',
 'fire_2001',
 'fire_2002',
 'fire_2003',
 'fire_2004',
 'fire_2005',
 'fire_2006',
 'fire_2007',
 'fire_2008',
 'fire_2009',
 'fire_2010',
 'fire_2011',
 'fire_2012',
 'fire_2013',
 'fire_2014',
 'fire_2015',
 'fire_2016',
 'fire_2017',
 'fire_2018',
 'fire_2019',
 'fire_2020',
 'fire_2021',
 'fire_2022',
 'AreaHa',
 'co2fire2003',
 'co2fire2004',
 'co2fire2005',
 'co2fire2006',
 'co2fire2007',
 'co2fire2008',
 'co2fire2009',
 'co2fire2010',
 'co2fire2011',
 'co2fire2012',
 'co2fire2013',
 'co2fire2014',
 'co2fire2015',
 'co2fire2016',
 'co2fire2017',
 'co2fire2018',
 'co2fire2019',
 'co2fire2020',
 'co2fire2021',
 'co2fire2022',
 'burntCO2_dev_2003',
 'burntCO2_dev_2004',
 'burntCO2_dev_2005',
 'burntCO2_dev_2006',
 'burntCO2_dev_2007',
 'burntCO2_dev_2008',
 'burntCO2_dev_2009',
 'burntCO2_dev_2010',
 'burntCO2_dev_2011',
 'burntCO

In [4]:
meltcols = ['GridNum10km',
 'LEVEL3_name',
 'LEVEL3_code',
 'ADM_ID',
 'ISO2',
 'protected_area',
 'env_zones',
 'LULUCF_CODE',
 'LULUCF_DESCRIPTION',
  'AreaHa', 'burntCO2_10km_mean',
 'burntCO2_10km_std',
 'burntCO2_10km_count','burntCO2_trend',
 'burntCO2_slope',
 'burntCO2_pval',
 'burntCO2_relch']

In [5]:
sel_cols = ['GridNum10km',
 'LEVEL3_name',
 'LEVEL3_code',
 'ADM_ID',
 'ISO2',
 'protected_area',
 'env_zones',
 'LULUCF_CODE',
 'LULUCF_DESCRIPTION',
 'fire_2000',
 'fire_2001',
 'fire_2002',
 'fire_2003',
 'fire_2004',
 'fire_2005',
 'fire_2006',
 'fire_2007',
 'fire_2008',
 'fire_2009',
 'fire_2010',
 'fire_2011',
 'fire_2012',
 'fire_2013',
 'fire_2014',
 'fire_2015',
 'fire_2016',
 'fire_2017',
 'fire_2018',
 'fire_2019',
 'fire_2020',
 'fire_2021',
 'fire_2022',
 'AreaHa', 'burntCO2_10km_mean',
 'burntCO2_10km_std',
 'burntCO2_10km_count','burntCO2_trend',
 'burntCO2_slope',
 'burntCO2_pval',
 'burntCO2_relch']
df_sel = fireco2.filter(sel_cols)
fires = df_sel.melt(id_vars=meltcols, var_name="year", value_name="FireHa")
fires['year'] = fires.year.str[-4:].astype(int)
fires.head()

,GridNum10km,LEVEL3_name,LEVEL3_code,ADM_ID,ISO2,protected_area,env_zones,LULUCF_CODE,LULUCF_DESCRIPTION,AreaHa,burntCO2_10km_mean,burntCO2_10km_std,burntCO2_10km_count,burntCO2_trend,burntCO2_slope,burntCO2_pval,burntCO2_relch,year,FireHa
0,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,None,ATN,FL,Forest land,37.0,0.785714,2.939874,16,0.0,0.000000,0.385547,NaN,2000,0
1,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,None,ATN,GL,Grassland,26.0,0.785714,2.939874,16,0.0,0.000000,0.385547,NaN,2000,0
2,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,Protected,ATN,FL,Forest land,1.0,0.785714,2.939874,16,0.0,0.000000,0.385547,NaN,2000,0
3,14435007124733952,Perth & Kinross and Stirling,UKM77,1360.0,GB,Protected,ATN,GL,Grassland,7.0,77.333336,107.319725,17,0.0,0.000000,0.540420,0.000000,2000,0
4,14451091777257472,East Lothian and Midlothian,UKM73,1357.0,GB,None,ATN,GL,Grassland,92.0,234.800003,258.019989,17,0.0,6.733333,0.364403,159.055115,2000,0


#### 2.2 Emissions

In [6]:
meltcols = ['GridNum10km',
 'LEVEL3_name',
 'LEVEL3_code',
 'ADM_ID',
 'ISO2',
 'protected_area',
 'env_zones',
 'LULUCF_CODE',
 'LULUCF_DESCRIPTION',
  'AreaHa']

In [7]:
sel_cols = ['GridNum10km',
 'LEVEL3_name',
 'LEVEL3_code',
 'ADM_ID',
 'ISO2',
 'protected_area',
 'env_zones',
 'LULUCF_CODE',
 'LULUCF_DESCRIPTION',
 'co2fire2003',
 'co2fire2004',
 'co2fire2005',
 'co2fire2006',
 'co2fire2007',
 'co2fire2008',
 'co2fire2009',
 'co2fire2010',
 'co2fire2011',
 'co2fire2012',
 'co2fire2013',
 'co2fire2014',
 'co2fire2015',
 'co2fire2016',
 'co2fire2017',
 'co2fire2018',
 'co2fire2019',
 'co2fire2020',
 'co2fire2021',
 'co2fire2022',
 'AreaHa']
df_sel = fireco2.filter(sel_cols)
co2fire = df_sel.melt(id_vars=meltcols, var_name="year", value_name="co2fireTn_10km")
co2fire['year'] = co2fire.year.str[-4:].astype(int)
co2fire.head()

,GridNum10km,LEVEL3_name,LEVEL3_code,ADM_ID,ISO2,protected_area,env_zones,LULUCF_CODE,LULUCF_DESCRIPTION,AreaHa,year,co2fireTn_10km
0,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,None,ATN,FL,Forest land,37.0,2003,863
1,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,None,ATN,GL,Grassland,26.0,2003,863
2,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,Protected,ATN,FL,Forest land,1.0,2003,863
3,14435007124733952,Perth & Kinross and Stirling,UKM77,1360.0,GB,Protected,ATN,GL,Grassland,7.0,2003,876
4,14451091777257472,East Lothian and Midlothian,UKM73,1357.0,GB,None,ATN,GL,Grassland,92.0,2003,1093


In [8]:
sel_cols = ['GridNum10km',
 'LEVEL3_name',
 'LEVEL3_code',
 'ADM_ID',
 'ISO2',
 'protected_area',
 'env_zones',
 'LULUCF_CODE',
 'LULUCF_DESCRIPTION',
 'co2fire2003',
 'co2fire2004',
 'co2fire2005',
 'co2fire2006',
 'co2fire2007',
 'co2fire2008',
 'co2fire2009',
 'co2fire2010',
 'co2fire2011',
 'co2fire2012',
 'co2fire2013',
 'co2fire2014',
 'co2fire2015',
 'co2fire2016',
 'co2fire2017',
 'co2fire2018',
 'co2fire2019',
 'co2fire2020',
 'co2fire2021',
 'co2fire2022',
 'AreaHa']
df_sel = fireco2.filter(sel_cols)
co2fire = df_sel.melt(id_vars=meltcols, var_name="year", value_name="co2fireTn_10km")
co2fire['year'] = co2fire.year.str[-4:].astype(int)
co2fire.head()

,GridNum10km,LEVEL3_name,LEVEL3_code,ADM_ID,ISO2,protected_area,env_zones,LULUCF_CODE,LULUCF_DESCRIPTION,AreaHa,year,co2fireTn_10km
0,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,None,ATN,FL,Forest land,37.0,2003,863
1,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,None,ATN,GL,Grassland,26.0,2003,863
2,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,Protected,ATN,FL,Forest land,1.0,2003,863
3,14435007124733952,Perth & Kinross and Stirling,UKM77,1360.0,GB,Protected,ATN,GL,Grassland,7.0,2003,876
4,14451091777257472,East Lothian and Midlothian,UKM73,1357.0,GB,None,ATN,GL,Grassland,92.0,2003,1093


In [9]:
# replace value 1 as it should be 0, artifact after reprojection and round up.
co2fire.co2fireTn_10km.replace(1, 0, inplace= True)

#### 2.3 Emissions deviation

In [10]:
sel_cols = ['GridNum10km',
 'LEVEL3_name',
 'LEVEL3_code',
 'ADM_ID',
 'ISO2',
 'protected_area',
 'env_zones',
 'LULUCF_CODE',
 'LULUCF_DESCRIPTION',
 'burntCO2_dev_2003',
 'burntCO2_dev_2004',
 'burntCO2_dev_2005',
 'burntCO2_dev_2006',
 'burntCO2_dev_2007',
 'burntCO2_dev_2008',
 'burntCO2_dev_2009',
 'burntCO2_dev_2010',
 'burntCO2_dev_2011',
 'burntCO2_dev_2012',
 'burntCO2_dev_2013',
 'burntCO2_dev_2014',
 'burntCO2_dev_2015',
 'burntCO2_dev_2016',
 'burntCO2_dev_2017',
 'burntCO2_dev_2018',
 'burntCO2_dev_2019',
 'burntCO2_dev_2020',
 'burntCO2_dev_2021',
 'burntCO2_dev_2022',
 'AreaHa']
df_sel = fireco2.filter(sel_cols)
burntCO2_dev = df_sel.melt(id_vars=meltcols, var_name="year", value_name="burntCO2_dev_10km")
burntCO2_dev['year'] = burntCO2_dev.year.str[-4:].astype(int)
burntCO2_dev.head()

,GridNum10km,LEVEL3_name,LEVEL3_code,ADM_ID,ISO2,protected_area,env_zones,LULUCF_CODE,LULUCF_DESCRIPTION,AreaHa,year,burntCO2_dev_10km
0,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,None,ATN,FL,Forest land,37.0,2003,NaN
1,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,None,ATN,GL,Grassland,26.0,2003,NaN
2,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,Protected,ATN,FL,Forest land,1.0,2003,NaN
3,14435007124733952,Perth & Kinross and Stirling,UKM77,1360.0,GB,Protected,ATN,GL,Grassland,7.0,2003,NaN
4,14451091777257472,East Lothian and Midlothian,UKM73,1357.0,GB,None,ATN,GL,Grassland,92.0,2003,NaN


In [11]:
burntCO2_dev.describe()

,GridNum10km,ADM_ID,AreaHa,year,burntCO2_dev_10km
count,1.048080e+06,967500.000000,967500.000000,1.048080e+06,832333.000000
mean,1.875279e+16,806.632062,186.354935,2.012500e+03,-28.397636
std,5.899324e+15,417.348798,471.260920,5.766284e+00,2117.841778
min,4.637216e+15,0.000000,1.000000,2.003000e+03,-79255.390625
25%,1.411054e+16,467.000000,10.000000,2.007750e+03,-222.866669
50%,1.870431e+16,742.000000,40.000000,2.012500e+03,-25.333334
75%,2.313506e+16,1067.000000,151.000000,2.017250e+03,76.428574
max,3.214322e+16,1604.000000,9231.000000,2.022000e+03,78990.625000


In [12]:
burntCO2_dev.query('GridNum10km ==4874753521221632 and year<2006')

,GridNum10km,LEVEL3_name,LEVEL3_code,ADM_ID,ISO2,protected_area,env_zones,LULUCF_CODE,LULUCF_DESCRIPTION,AreaHa,year,burntCO2_dev_10km
3553,4874753521221632,La Palma,ES707,1443.0,ES,Protected,MAC,OL,Other land,392.0,2003,-1543.357178
3554,4874753521221632,La Palma,ES707,1443.0,ES,Protected,MAC,CL,Cropland,52.0,2003,-1543.357178
3555,4874753521221632,La Palma,ES707,1443.0,ES,None,MAC,OL,Other land,66.0,2003,-1543.357178
3556,4874753521221632,La Palma,ES707,1443.0,ES,None,MAC,CL,Cropland,58.0,2003,-1543.357178
3557,4874753521221632,La Palma,ES707,1443.0,ES,None,MAC,SL,Settlements,25.0,2003,-1543.357178
3558,4874753521221632,La Palma,ES707,1443.0,ES,Protected,MAC,FL,Forest land,8.0,2003,-1543.357178
3559,4874753521221632,La Palma,ES707,1443.0,ES,Protected,MAC,GL,Grassland,2.0,2003,-1543.357178
3560,4874753521221632,La Palma,ES707,1443.0,ES,None,MAC,FL,Forest land,3.0,2003,-1543.357178
55957,4874753521221632,La Palma,ES707,1443.0,ES,Protected,MAC,OL,Other land,392.0,2004,-1543.357178
55958,4874753521221632,La Palma,ES707,1443.0,ES,Protected,MAC,CL,Cropland,52.0,2004,-1543.357178


#### 2.4 Emissions anomalies

In [13]:
meltcols = ['GridNum10km',
 'LEVEL3_name',
 'LEVEL3_code',
 'ADM_ID',
 'ISO2',
 'protected_area',
 'env_zones',
 'LULUCF_CODE',
 'LULUCF_DESCRIPTION',
  'AreaHa']

In [14]:
sel_cols = ['GridNum10km',
 'LEVEL3_name',
 'LEVEL3_code',
 'ADM_ID',
 'ISO2',
 'protected_area',
 'env_zones',
 'LULUCF_CODE',
 'LULUCF_DESCRIPTION',
 'burntCO2_anom_2003',
 'burntCO2_anom_2004',
 'burntCO2_anom_2005',
 'burntCO2_anom_2006',
 'burntCO2_anom_2007',
 'burntCO2_anom_2008',
 'burntCO2_anom_2009',
 'burntCO2_anom_2010',
 'burntCO2_anom_2011',
 'burntCO2_anom_2012',
 'burntCO2_anom_2013',
 'burntCO2_anom_2014',
 'burntCO2_anom_2015',
 'burntCO2_anom_2016',
 'burntCO2_anom_2017',
 'burntCO2_anom_2018',
 'burntCO2_anom_2019',
 'burntCO2_anom_2020',
 'burntCO2_anom_2021',
 'burntCO2_anom_2022',
 'AreaHa']
df_sel = fireco2.filter(sel_cols)
burntCO2_anom = df_sel.melt(id_vars=meltcols, var_name="year", value_name="burntCO2_anom_10km")
burntCO2_anom['year'] = burntCO2_anom.year.str[-4:].astype(int)
burntCO2_anom.head()

,GridNum10km,LEVEL3_name,LEVEL3_code,ADM_ID,ISO2,protected_area,env_zones,LULUCF_CODE,LULUCF_DESCRIPTION,AreaHa,year,burntCO2_anom_10km
0,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,None,ATN,FL,Forest land,37.0,2003,NaN
1,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,None,ATN,GL,Grassland,26.0,2003,NaN
2,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,Protected,ATN,FL,Forest land,1.0,2003,NaN
3,14435007124733952,Perth & Kinross and Stirling,UKM77,1360.0,GB,Protected,ATN,GL,Grassland,7.0,2003,NaN
4,14451091777257472,East Lothian and Midlothian,UKM73,1357.0,GB,None,ATN,GL,Grassland,92.0,2003,NaN


In [15]:
burntCO2_anom.describe()

,GridNum10km,ADM_ID,AreaHa,year,burntCO2_anom_10km
count,1.048080e+06,967500.000000,967500.000000,1.048080e+06,808940.000000
mean,1.875279e+16,806.632062,186.354935,2.012500e+03,0.001157
std,5.899324e+15,417.348798,471.260920,5.766284e+00,1.088977
min,4.637216e+15,0.000000,1.000000,2.003000e+03,-2.477560
25%,1.411054e+16,467.000000,10.000000,2.007750e+03,-0.674162
50%,1.870431e+16,742.000000,40.000000,2.012500e+03,-0.372040
75%,2.313506e+16,1067.000000,151.000000,2.017250e+03,0.500182
max,3.214322e+16,1604.000000,9231.000000,2.022000e+03,80.801979


#### 2.5 Emissions trends

In [16]:
sel_cols = ['GridNum10km',
 'LEVEL3_name',
 'LEVEL3_code',
 'ADM_ID',
 'ISO2',
 'protected_area',
 'env_zones',
 'LULUCF_CODE',
 'LULUCF_DESCRIPTION',
 'burntCO2_trend',
 'burntCO2_slope',
 'burntCO2_pval',
 'burntCO2_relch',
 'AreaHa']
burntCO2_trends = fireco2.filter(sel_cols)
burntCO2_trends.head()

,GridNum10km,LEVEL3_name,LEVEL3_code,ADM_ID,ISO2,protected_area,env_zones,LULUCF_CODE,LULUCF_DESCRIPTION,burntCO2_trend,burntCO2_slope,burntCO2_pval,burntCO2_relch,AreaHa
0,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,None,ATN,FL,Forest land,0.0,0.000000,0.385547,NaN,37.0
1,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,None,ATN,GL,Grassland,0.0,0.000000,0.385547,NaN,26.0
2,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,Protected,ATN,FL,Forest land,0.0,0.000000,0.385547,NaN,1.0
3,14435007124733952,Perth & Kinross and Stirling,UKM77,1360.0,GB,Protected,ATN,GL,Grassland,0.0,0.000000,0.540420,0.000000,7.0
4,14451091777257472,East Lothian and Midlothian,UKM73,1357.0,GB,None,ATN,GL,Grassland,0.0,6.733333,0.364403,159.055115,92.0


### 3.. Weighted averages

#### 3.1 Fires

In [17]:
fires.head()

,GridNum10km,LEVEL3_name,LEVEL3_code,ADM_ID,ISO2,protected_area,env_zones,LULUCF_CODE,LULUCF_DESCRIPTION,AreaHa,burntCO2_10km_mean,burntCO2_10km_std,burntCO2_10km_count,burntCO2_trend,burntCO2_slope,burntCO2_pval,burntCO2_relch,year,FireHa
0,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,None,ATN,FL,Forest land,37.0,0.785714,2.939874,16,0.0,0.000000,0.385547,NaN,2000,0
1,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,None,ATN,GL,Grassland,26.0,0.785714,2.939874,16,0.0,0.000000,0.385547,NaN,2000,0
2,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,Protected,ATN,FL,Forest land,1.0,0.785714,2.939874,16,0.0,0.000000,0.385547,NaN,2000,0
3,14435007124733952,Perth & Kinross and Stirling,UKM77,1360.0,GB,Protected,ATN,GL,Grassland,7.0,77.333336,107.319725,17,0.0,0.000000,0.540420,0.000000,2000,0
4,14451091777257472,East Lothian and Midlothian,UKM73,1357.0,GB,None,ATN,GL,Grassland,92.0,234.800003,258.019989,17,0.0,6.733333,0.364403,159.055115,2000,0


In [18]:
## Compute total Fire area - agregating by RefGrid 10 km and year
grcols = ['GridNum10km', 'year']
fires_grouped_FTA = fires.groupby(grcols).agg(FireHa_10km = ('FireHa', 'sum')).reset_index()
fires_grouped_FTA.head(20)

,GridNum10km,year,FireHa_10km
0,4637216059949056,2000,0
1,4637216059949056,2001,0
2,4637216059949056,2002,0
3,4637216059949056,2003,0
4,4637216059949056,2004,0
5,4637216059949056,2005,0
6,4637216059949056,2006,0
7,4637216059949056,2007,3077
8,4637216059949056,2008,0
9,4637216059949056,2009,0


In [19]:
fires_FTA = fires.merge(fires_grouped_FTA, on= grcols)
fires_FTA.query('year==2007 and  GridNum10km==4637216059949056')

,GridNum10km,LEVEL3_name,LEVEL3_code,ADM_ID,ISO2,protected_area,env_zones,LULUCF_CODE,LULUCF_DESCRIPTION,AreaHa,burntCO2_10km_mean,burntCO2_10km_std,burntCO2_10km_count,burntCO2_trend,burntCO2_slope,burntCO2_pval,burntCO2_relch,year,FireHa,FireHa_10km
399994,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,OL,Other land,1087.0,120.666664,269.516418,17,0.0,0.0,0.79717,0.0,2007,1087,3077
399995,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,FL,Forest land,723.0,120.666664,269.516418,17,0.0,0.0,0.79717,0.0,2007,723,3077
399996,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,OL,Other land,929.0,120.666664,269.516418,17,0.0,0.0,0.79717,0.0,2007,929,3077
399997,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,GL,Grassland,97.0,120.666664,269.516418,17,0.0,0.0,0.79717,0.0,2007,97,3077
399998,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,FL,Forest land,72.0,120.666664,269.516418,17,0.0,0.0,0.79717,0.0,2007,72,3077
399999,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,CL,Cropland,113.0,120.666664,269.516418,17,0.0,0.0,0.79717,0.0,2007,113,3077
400000,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,WL,Wetlands,14.0,120.666664,269.516418,17,0.0,0.0,0.79717,0.0,2007,14,3077
400001,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,GL,Grassland,38.0,120.666664,269.516418,17,0.0,0.0,0.79717,0.0,2007,38,3077
400002,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,CL,Cropland,4.0,120.666664,269.516418,17,0.0,0.0,0.79717,0.0,2007,4,3077


In [20]:
fires_FTA.query('year==2007 and GridNum10km==4637216059949056').FireHa.sum()

3077

#### 3.2 Emissions

In [21]:
co2fire.query('year==2007 and GridNum10km==4637216059949056')

,GridNum10km,LEVEL3_name,LEVEL3_code,ADM_ID,ISO2,protected_area,env_zones,LULUCF_CODE,LULUCF_DESCRIPTION,AreaHa,year,co2fireTn_10km
242777,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,OL,Other land,1087.0,2007,58026
242778,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,FL,Forest land,723.0,2007,58026
242779,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,OL,Other land,929.0,2007,58026
242780,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,GL,Grassland,97.0,2007,58026
242781,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,FL,Forest land,72.0,2007,58026
242782,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,CL,Cropland,113.0,2007,58026
242783,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,WL,Wetlands,14.0,2007,58026
242784,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,GL,Grassland,38.0,2007,58026
242785,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,CL,Cropland,4.0,2007,58026


In [22]:
mergecols = ['GridNum10km',
 'LEVEL3_name',
 'LEVEL3_code',
 'ADM_ID',
 'ISO2',
 'protected_area',
 'env_zones',
 'LULUCF_CODE',
 'LULUCF_DESCRIPTION',
 'AreaHa', 'year']
fires_FTA_co2fire = fires_FTA.merge(co2fire, on= mergecols)
fires_FTA_co2fire.query('year==2007 and GridNum10km==4637216059949056')

,GridNum10km,LEVEL3_name,LEVEL3_code,ADM_ID,ISO2,protected_area,env_zones,LULUCF_CODE,LULUCF_DESCRIPTION,AreaHa,...,burntCO2_10km_std,burntCO2_10km_count,burntCO2_trend,burntCO2_slope,burntCO2_pval,burntCO2_relch,year,FireHa,FireHa_10km,co2fireTn_10km
242782,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,OL,Other land,1087.0,...,269.516418,17,0.0,0.0,0.79717,0.0,2007,1087,3077,58026
242783,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,FL,Forest land,723.0,...,269.516418,17,0.0,0.0,0.79717,0.0,2007,723,3077,58026
242784,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,OL,Other land,929.0,...,269.516418,17,0.0,0.0,0.79717,0.0,2007,929,3077,58026
242785,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,GL,Grassland,97.0,...,269.516418,17,0.0,0.0,0.79717,0.0,2007,97,3077,58026
242786,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,FL,Forest land,72.0,...,269.516418,17,0.0,0.0,0.79717,0.0,2007,72,3077,58026
242787,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,CL,Cropland,113.0,...,269.516418,17,0.0,0.0,0.79717,0.0,2007,113,3077,58026
242788,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,WL,Wetlands,14.0,...,269.516418,17,0.0,0.0,0.79717,0.0,2007,14,3077,58026
242789,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,GL,Grassland,38.0,...,269.516418,17,0.0,0.0,0.79717,0.0,2007,38,3077,58026
242790,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,CL,Cropland,4.0,...,269.516418,17,0.0,0.0,0.79717,0.0,2007,4,3077,58026


In [23]:
fires_FTA_co2fire['co2fireTn'] = round((fires_FTA_co2fire.co2fireTn_10km * fires_FTA_co2fire.FireHa)/fires_FTA_co2fire.FireHa_10km,3)
fires_FTA_co2fire.co2fireTn.fillna(0, inplace=True)

In [24]:
fires_FTA_co2fire.query('year==2007 and GridNum10km==4637216059949056')

,GridNum10km,LEVEL3_name,LEVEL3_code,ADM_ID,ISO2,protected_area,env_zones,LULUCF_CODE,LULUCF_DESCRIPTION,AreaHa,...,burntCO2_10km_count,burntCO2_trend,burntCO2_slope,burntCO2_pval,burntCO2_relch,year,FireHa,FireHa_10km,co2fireTn_10km,co2fireTn
242782,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,OL,Other land,1087.0,...,17,0.0,0.0,0.79717,0.0,2007,1087,3077,58026,20498.623
242783,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,FL,Forest land,723.0,...,17,0.0,0.0,0.79717,0.0,2007,723,3077,58026,13634.318
242784,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,OL,Other land,929.0,...,17,0.0,0.0,0.79717,0.0,2007,929,3077,58026,17519.062
242785,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,GL,Grassland,97.0,...,17,0.0,0.0,0.79717,0.0,2007,97,3077,58026,1829.224
242786,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,FL,Forest land,72.0,...,17,0.0,0.0,0.79717,0.0,2007,72,3077,58026,1357.774
242787,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,CL,Cropland,113.0,...,17,0.0,0.0,0.79717,0.0,2007,113,3077,58026,2130.952
242788,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,WL,Wetlands,14.0,...,17,0.0,0.0,0.79717,0.0,2007,14,3077,58026,264.012
242789,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,GL,Grassland,38.0,...,17,0.0,0.0,0.79717,0.0,2007,38,3077,58026,716.603
242790,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,CL,Cropland,4.0,...,17,0.0,0.0,0.79717,0.0,2007,4,3077,58026,75.432


In [25]:
fires_FTA_co2fire.head()

,GridNum10km,LEVEL3_name,LEVEL3_code,ADM_ID,ISO2,protected_area,env_zones,LULUCF_CODE,LULUCF_DESCRIPTION,AreaHa,...,burntCO2_10km_count,burntCO2_trend,burntCO2_slope,burntCO2_pval,burntCO2_relch,year,FireHa,FireHa_10km,co2fireTn_10km,co2fireTn
0,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,None,ATN,FL,Forest land,37.0,...,16,0.0,0.0,0.385547,NaN,2003,0,739,863,0.000
1,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,None,ATN,GL,Grassland,26.0,...,16,0.0,0.0,0.385547,NaN,2003,0,739,863,0.000
2,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,Protected,ATN,FL,Forest land,1.0,...,16,0.0,0.0,0.385547,NaN,2003,0,739,863,0.000
3,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,Protected,ATN,WL,Wetlands,724.0,...,16,0.0,0.0,0.385547,NaN,2003,722,739,863,843.147
4,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,Protected,ATN,GL,Grassland,81.0,...,16,0.0,0.0,0.385547,NaN,2003,17,739,863,19.853


In [26]:
fires_FTA_co2fire.describe()

,GridNum10km,ADM_ID,AreaHa,burntCO2_10km_mean,burntCO2_10km_std,burntCO2_10km_count,burntCO2_trend,burntCO2_slope,burntCO2_pval,burntCO2_relch,year,FireHa,FireHa_10km,co2fireTn_10km,co2fireTn
count,1.048080e+06,967500.000000,967500.000000,1.002880e+06,1.002880e+06,1.048080e+06,976280.000000,976280.000000,972940.000000,778120.000000,1.048080e+06,1.048080e+06,1.048080e+06,1.048080e+06,1.048080e+06
mean,1.875279e+16,806.632062,186.354935,9.138963e+02,8.249078e+02,1.588301e+01,0.003892,-13.321739,0.443203,240.185751,2.012500e+03,1.172532e+01,6.958050e+01,2.502785e+03,2.763709e+02
std,5.899324e+15,417.348798,471.260920,3.066085e+03,1.948929e+03,3.612500e+00,0.323967,95.950120,0.311480,2707.593726,5.766284e+00,1.119533e+02,3.838628e+02,2.243900e+04,3.547859e+03
min,4.637216e+15,0.000000,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000,-3610.000000,0.001007,-285.714294,2.003000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.411054e+16,467.000000,10.000000,5.286666e+01,7.579433e+01,1.600000e+01,0.000000,-6.460417,0.157109,-57.058002,2.007750e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.870431e+16,742.000000,40.000000,2.283333e+02,2.520641e+02,1.600000e+01,0.000000,0.000000,0.404670,0.000000,2.012500e+03,0.000000e+00,0.000000e+00,1.750000e+02,0.000000e+00
75%,2.313506e+16,1067.000000,151.000000,7.546250e+02,6.891245e+02,1.700000e+01,0.000000,1.805556,0.710836,83.922363,2.017250e+03,0.000000e+00,0.000000e+00,1.031000e+03,0.000000e+00
max,3.214322e+16,1604.000000,9231.000000,1.494064e+05,4.179039e+04,2.000000e+01,1.000000,769.642883,1.000000,276000.000000,2.022000e+03,9.112000e+03,1.000000e+04,4.982198e+06,5.207385e+05


#### 3.3 Emissions deviation

In [27]:
burntCO2_dev.query('year==2007 and GridNum10km==4637216059949056')

,GridNum10km,LEVEL3_name,LEVEL3_code,ADM_ID,ISO2,protected_area,env_zones,LULUCF_CODE,LULUCF_DESCRIPTION,AreaHa,year,burntCO2_dev_10km
242777,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,OL,Other land,1087.0,2007,NaN
242778,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,FL,Forest land,723.0,2007,NaN
242779,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,OL,Other land,929.0,2007,NaN
242780,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,GL,Grassland,97.0,2007,NaN
242781,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,FL,Forest land,72.0,2007,NaN
242782,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,CL,Cropland,113.0,2007,NaN
242783,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,WL,Wetlands,14.0,2007,NaN
242784,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,GL,Grassland,38.0,2007,NaN
242785,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,CL,Cropland,4.0,2007,NaN


In [28]:
## merge deviation values 
fires_FTA_co2fire_dev = fires_FTA_co2fire.merge(burntCO2_dev, on= mergecols)
fires_FTA_co2fire_dev.query('year==2007 and GridNum10km==4637216059949056')

,GridNum10km,LEVEL3_name,LEVEL3_code,ADM_ID,ISO2,protected_area,env_zones,LULUCF_CODE,LULUCF_DESCRIPTION,AreaHa,...,burntCO2_trend,burntCO2_slope,burntCO2_pval,burntCO2_relch,year,FireHa,FireHa_10km,co2fireTn_10km,co2fireTn,burntCO2_dev_10km
242782,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,OL,Other land,1087.0,...,0.0,0.0,0.79717,0.0,2007,1087,3077,58026,20498.623,NaN
242783,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,FL,Forest land,723.0,...,0.0,0.0,0.79717,0.0,2007,723,3077,58026,13634.318,NaN
242784,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,OL,Other land,929.0,...,0.0,0.0,0.79717,0.0,2007,929,3077,58026,17519.062,NaN
242785,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,GL,Grassland,97.0,...,0.0,0.0,0.79717,0.0,2007,97,3077,58026,1829.224,NaN
242786,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,FL,Forest land,72.0,...,0.0,0.0,0.79717,0.0,2007,72,3077,58026,1357.774,NaN
242787,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,CL,Cropland,113.0,...,0.0,0.0,0.79717,0.0,2007,113,3077,58026,2130.952,NaN
242788,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,WL,Wetlands,14.0,...,0.0,0.0,0.79717,0.0,2007,14,3077,58026,264.012,NaN
242789,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,GL,Grassland,38.0,...,0.0,0.0,0.79717,0.0,2007,38,3077,58026,716.603,NaN
242790,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,CL,Cropland,4.0,...,0.0,0.0,0.79717,0.0,2007,4,3077,58026,75.432,NaN


In [29]:
### compute deviations values for 100m unique combinations
fires_FTA_co2fire_dev['burntCO2_dev'] = round((fires_FTA_co2fire_dev.burntCO2_dev_10km * fires_FTA_co2fire_dev.FireHa)/fires_FTA_co2fire_dev.FireHa_10km,3)
fires_FTA_co2fire_dev.burntCO2_dev.fillna(0, inplace=True)

In [30]:
fires_FTA_co2fire_dev.describe()

,GridNum10km,ADM_ID,AreaHa,burntCO2_10km_mean,burntCO2_10km_std,burntCO2_10km_count,burntCO2_trend,burntCO2_slope,burntCO2_pval,burntCO2_relch,year,FireHa,FireHa_10km,co2fireTn_10km,co2fireTn,burntCO2_dev_10km,burntCO2_dev
count,1.048080e+06,967500.000000,967500.000000,1.002880e+06,1.002880e+06,1.048080e+06,976280.000000,976280.000000,972940.000000,778120.000000,1.048080e+06,1.048080e+06,1.048080e+06,1.048080e+06,1.048080e+06,832333.000000,1.048080e+06
mean,1.875279e+16,806.632062,186.354935,9.138963e+02,8.249078e+02,1.588301e+01,0.003892,-13.321739,0.443203,240.185751,2.012500e+03,1.172532e+01,6.958050e+01,2.502785e+03,2.763709e+02,-28.397636,1.270344e+01
std,5.899324e+15,417.348798,471.260920,3.066085e+03,1.948929e+03,3.612500e+00,0.323967,95.950120,0.311480,2707.593726,5.766284e+00,1.119533e+02,3.838628e+02,2.243900e+04,3.547859e+03,2117.841778,4.209165e+02
min,4.637216e+15,0.000000,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000,-3610.000000,0.001007,-285.714294,2.003000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-79255.390625,-4.425713e+04
25%,1.411054e+16,467.000000,10.000000,5.286666e+01,7.579433e+01,1.600000e+01,0.000000,-6.460417,0.157109,-57.058002,2.007750e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-222.866669,0.000000e+00
50%,1.870431e+16,742.000000,40.000000,2.283333e+02,2.520641e+02,1.600000e+01,0.000000,0.000000,0.404670,0.000000,2.012500e+03,0.000000e+00,0.000000e+00,1.750000e+02,0.000000e+00,-25.333334,0.000000e+00
75%,2.313506e+16,1067.000000,151.000000,7.546250e+02,6.891245e+02,1.700000e+01,0.000000,1.805556,0.710836,83.922363,2.017250e+03,0.000000e+00,0.000000e+00,1.031000e+03,0.000000e+00,76.428574,0.000000e+00
max,3.214322e+16,1604.000000,9231.000000,1.494064e+05,4.179039e+04,2.000000e+01,1.000000,769.642883,1.000000,276000.000000,2.022000e+03,9.112000e+03,1.000000e+04,4.982198e+06,5.207385e+05,78990.625000,4.178266e+04


In [31]:
fires_FTA_co2fire_dev.describe()

,GridNum10km,ADM_ID,AreaHa,burntCO2_10km_mean,burntCO2_10km_std,burntCO2_10km_count,burntCO2_trend,burntCO2_slope,burntCO2_pval,burntCO2_relch,year,FireHa,FireHa_10km,co2fireTn_10km,co2fireTn,burntCO2_dev_10km,burntCO2_dev
count,1.048080e+06,967500.000000,967500.000000,1.002880e+06,1.002880e+06,1.048080e+06,976280.000000,976280.000000,972940.000000,778120.000000,1.048080e+06,1.048080e+06,1.048080e+06,1.048080e+06,1.048080e+06,832333.000000,1.048080e+06
mean,1.875279e+16,806.632062,186.354935,9.138963e+02,8.249078e+02,1.588301e+01,0.003892,-13.321739,0.443203,240.185751,2.012500e+03,1.172532e+01,6.958050e+01,2.502785e+03,2.763709e+02,-28.397636,1.270344e+01
std,5.899324e+15,417.348798,471.260920,3.066085e+03,1.948929e+03,3.612500e+00,0.323967,95.950120,0.311480,2707.593726,5.766284e+00,1.119533e+02,3.838628e+02,2.243900e+04,3.547859e+03,2117.841778,4.209165e+02
min,4.637216e+15,0.000000,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000,-3610.000000,0.001007,-285.714294,2.003000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-79255.390625,-4.425713e+04
25%,1.411054e+16,467.000000,10.000000,5.286666e+01,7.579433e+01,1.600000e+01,0.000000,-6.460417,0.157109,-57.058002,2.007750e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-222.866669,0.000000e+00
50%,1.870431e+16,742.000000,40.000000,2.283333e+02,2.520641e+02,1.600000e+01,0.000000,0.000000,0.404670,0.000000,2.012500e+03,0.000000e+00,0.000000e+00,1.750000e+02,0.000000e+00,-25.333334,0.000000e+00
75%,2.313506e+16,1067.000000,151.000000,7.546250e+02,6.891245e+02,1.700000e+01,0.000000,1.805556,0.710836,83.922363,2.017250e+03,0.000000e+00,0.000000e+00,1.031000e+03,0.000000e+00,76.428574,0.000000e+00
max,3.214322e+16,1604.000000,9231.000000,1.494064e+05,4.179039e+04,2.000000e+01,1.000000,769.642883,1.000000,276000.000000,2.022000e+03,9.112000e+03,1.000000e+04,4.982198e+06,5.207385e+05,78990.625000,4.178266e+04


In [32]:
fires_FTA_co2fire_dev.head()

,GridNum10km,LEVEL3_name,LEVEL3_code,ADM_ID,ISO2,protected_area,env_zones,LULUCF_CODE,LULUCF_DESCRIPTION,AreaHa,...,burntCO2_slope,burntCO2_pval,burntCO2_relch,year,FireHa,FireHa_10km,co2fireTn_10km,co2fireTn,burntCO2_dev_10km,burntCO2_dev
0,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,None,ATN,FL,Forest land,37.0,...,0.0,0.385547,NaN,2003,0,739,863,0.000,NaN,0.0
1,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,None,ATN,GL,Grassland,26.0,...,0.0,0.385547,NaN,2003,0,739,863,0.000,NaN,0.0
2,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,Protected,ATN,FL,Forest land,1.0,...,0.0,0.385547,NaN,2003,0,739,863,0.000,NaN,0.0
3,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,Protected,ATN,WL,Wetlands,724.0,...,0.0,0.385547,NaN,2003,722,739,863,843.147,NaN,0.0
4,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,Protected,ATN,GL,Grassland,81.0,...,0.0,0.385547,NaN,2003,17,739,863,19.853,NaN,0.0


#### 3.4 Emissions anomalies

In [33]:
## merge deviation values 
fires_FTA_co2fire_dev_anom = fires_FTA_co2fire_dev.merge(burntCO2_anom, on= mergecols)
fires_FTA_co2fire_dev_anom.query('year==2007 and GridNum10km==4637216059949056')

,GridNum10km,LEVEL3_name,LEVEL3_code,ADM_ID,ISO2,protected_area,env_zones,LULUCF_CODE,LULUCF_DESCRIPTION,AreaHa,...,burntCO2_pval,burntCO2_relch,year,FireHa,FireHa_10km,co2fireTn_10km,co2fireTn,burntCO2_dev_10km,burntCO2_dev,burntCO2_anom_10km
242782,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,OL,Other land,1087.0,...,0.79717,0.0,2007,1087,3077,58026,20498.623,NaN,0.0,NaN
242783,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,FL,Forest land,723.0,...,0.79717,0.0,2007,723,3077,58026,13634.318,NaN,0.0,NaN
242784,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,OL,Other land,929.0,...,0.79717,0.0,2007,929,3077,58026,17519.062,NaN,0.0,NaN
242785,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,GL,Grassland,97.0,...,0.79717,0.0,2007,97,3077,58026,1829.224,NaN,0.0,NaN
242786,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,FL,Forest land,72.0,...,0.79717,0.0,2007,72,3077,58026,1357.774,NaN,0.0,NaN
242787,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,CL,Cropland,113.0,...,0.79717,0.0,2007,113,3077,58026,2130.952,NaN,0.0,NaN
242788,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,WL,Wetlands,14.0,...,0.79717,0.0,2007,14,3077,58026,264.012,NaN,0.0,NaN
242789,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,GL,Grassland,38.0,...,0.79717,0.0,2007,38,3077,58026,716.603,NaN,0.0,NaN
242790,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,CL,Cropland,4.0,...,0.79717,0.0,2007,4,3077,58026,75.432,NaN,0.0,NaN


In [34]:
### compute deviations values for 100m unique combinations
fires_FTA_co2fire_dev_anom['burntCO2_anom'] = round((fires_FTA_co2fire_dev_anom.burntCO2_anom_10km * fires_FTA_co2fire_dev_anom.FireHa)/fires_FTA_co2fire_dev_anom.FireHa_10km,3)
fires_FTA_co2fire_dev_anom.burntCO2_anom.fillna(0, inplace=True)

In [35]:
fires_FTA_co2fire_dev_anom.query('year==2007 and GridNum10km==4637216059949056')

,GridNum10km,LEVEL3_name,LEVEL3_code,ADM_ID,ISO2,protected_area,env_zones,LULUCF_CODE,LULUCF_DESCRIPTION,AreaHa,...,burntCO2_relch,year,FireHa,FireHa_10km,co2fireTn_10km,co2fireTn,burntCO2_dev_10km,burntCO2_dev,burntCO2_anom_10km,burntCO2_anom
242782,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,OL,Other land,1087.0,...,0.0,2007,1087,3077,58026,20498.623,NaN,0.0,NaN,0.0
242783,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,FL,Forest land,723.0,...,0.0,2007,723,3077,58026,13634.318,NaN,0.0,NaN,0.0
242784,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,OL,Other land,929.0,...,0.0,2007,929,3077,58026,17519.062,NaN,0.0,NaN,0.0
242785,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,GL,Grassland,97.0,...,0.0,2007,97,3077,58026,1829.224,NaN,0.0,NaN,0.0
242786,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,FL,Forest land,72.0,...,0.0,2007,72,3077,58026,1357.774,NaN,0.0,NaN,0.0
242787,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,CL,Cropland,113.0,...,0.0,2007,113,3077,58026,2130.952,NaN,0.0,NaN,0.0
242788,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,WL,Wetlands,14.0,...,0.0,2007,14,3077,58026,264.012,NaN,0.0,NaN,0.0
242789,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,GL,Grassland,38.0,...,0.0,2007,38,3077,58026,716.603,NaN,0.0,NaN,0.0
242790,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,CL,Cropland,4.0,...,0.0,2007,4,3077,58026,75.432,NaN,0.0,NaN,0.0


#### 3.5 Emissions stats

In [36]:
list(fires_FTA_co2fire_dev_anom.columns)

['GridNum10km',
 'LEVEL3_name',
 'LEVEL3_code',
 'ADM_ID',
 'ISO2',
 'protected_area',
 'env_zones',
 'LULUCF_CODE',
 'LULUCF_DESCRIPTION',
 'AreaHa',
 'burntCO2_10km_mean',
 'burntCO2_10km_std',
 'burntCO2_10km_count',
 'burntCO2_trend',
 'burntCO2_slope',
 'burntCO2_pval',
 'burntCO2_relch',
 'year',
 'FireHa',
 'FireHa_10km',
 'co2fireTn_10km',
 'co2fireTn',
 'burntCO2_dev_10km',
 'burntCO2_dev',
 'burntCO2_anom_10km',
 'burntCO2_anom']

In [37]:
### compute deviations values for 100m unique combinations
fires_FTA_co2fire_dev_anom['burntCO2_mean'] =  round((fires_FTA_co2fire_dev_anom.burntCO2_10km_mean * fires_FTA_co2fire_dev_anom.FireHa)/fires_FTA_co2fire_dev_anom.FireHa_10km,3)
fires_FTA_co2fire_dev_anom['burntCO2_std'] =  round((fires_FTA_co2fire_dev_anom.burntCO2_10km_std * fires_FTA_co2fire_dev_anom.FireHa)/fires_FTA_co2fire_dev_anom.FireHa_10km,3)


In [58]:
fires_FTA_co2fire_dev_anom.query('year==2007 and GridNum10km==4637216059949056')

,GridNum10km,LEVEL3_name,LEVEL3_code,ADM_ID,ISO2,protected_area,env_zones,LULUCF_CODE,LULUCF_DESCRIPTION,AreaHa,...,co2fireTn,burntCO2_dev_10km,burntCO2_dev,burntCO2_anom_10km,burntCO2_anom,burntCO2_mean,burntCO2_std,burntCO2_slope_ts,burntCO2_pval_ts,burntCO2_relchg
242782,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,OL,Other land,1087.0,...,20498.623,NaN,0.0,NaN,0.0,42.627,95.211,0.0,0.28161,0.0
242783,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,FL,Forest land,723.0,...,13634.318,NaN,0.0,NaN,0.0,28.353,63.328,0.0,0.18731,0.0
242784,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,OL,Other land,929.0,...,17519.062,NaN,0.0,NaN,0.0,36.431,81.372,0.0,0.24068,0.0
242785,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,GL,Grassland,97.0,...,1829.224,NaN,0.0,NaN,0.0,3.804,8.496,0.0,0.02513,0.0
242786,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,FL,Forest land,72.0,...,1357.774,NaN,0.0,NaN,0.0,2.824,6.307,0.0,0.01865,0.0
242787,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,CL,Cropland,113.0,...,2130.952,NaN,0.0,NaN,0.0,4.431,9.898,0.0,0.02928,0.0
242788,4637216059949056,Gran Canaria,ES705,371.0,ES,None,MAC,WL,Wetlands,14.0,...,264.012,NaN,0.0,NaN,0.0,0.549,1.226,0.0,0.00363,0.0
242789,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,GL,Grassland,38.0,...,716.603,NaN,0.0,NaN,0.0,1.490,3.328,0.0,0.00984,0.0
242790,4637216059949056,Gran Canaria,ES705,371.0,ES,Protected,MAC,CL,Cropland,4.0,...,75.432,NaN,0.0,NaN,0.0,0.157,0.350,0.0,0.00104,0.0


#### 3.6 Emissions trends

In [46]:
fires_FTA_co2fire_dev_anom['burntCO2_slope_ts'] = round((fires_FTA_co2fire_dev_anom.burntCO2_slope * fires_FTA_co2fire_dev_anom.FireHa)/fires_FTA_co2fire_dev_anom.FireHa_10km,3)
fires_FTA_co2fire_dev_anom['burntCO2_pval_ts'] = round((fires_FTA_co2fire_dev_anom.burntCO2_pval * fires_FTA_co2fire_dev_anom.FireHa)/fires_FTA_co2fire_dev_anom.FireHa_10km,5)
fires_FTA_co2fire_dev_anom['burntCO2_relchg'] = round((fires_FTA_co2fire_dev_anom.burntCO2_relch * fires_FTA_co2fire_dev_anom.FireHa)/fires_FTA_co2fire_dev_anom.FireHa_10km,5)

In [54]:
fires_FTA_co2fire_dev_anom.burntCO2_slope_ts.fillna(0, inplace=True)
fires_FTA_co2fire_dev_anom.burntCO2_pval_ts.fillna(0, inplace=True)
fires_FTA_co2fire_dev_anom.burntCO2_relchg.fillna(0, inplace=True)

### 4.. Prepare output tables

In [47]:
list(fires_FTA_co2fire_dev_anom.columns)

['GridNum10km',
 'LEVEL3_name',
 'LEVEL3_code',
 'ADM_ID',
 'ISO2',
 'protected_area',
 'env_zones',
 'LULUCF_CODE',
 'LULUCF_DESCRIPTION',
 'AreaHa',
 'burntCO2_10km_mean',
 'burntCO2_10km_std',
 'burntCO2_10km_count',
 'burntCO2_trend',
 'burntCO2_slope',
 'burntCO2_pval',
 'burntCO2_relch',
 'year',
 'FireHa',
 'FireHa_10km',
 'co2fireTn_10km',
 'co2fireTn',
 'burntCO2_dev_10km',
 'burntCO2_dev',
 'burntCO2_anom_10km',
 'burntCO2_anom',
 'burntCO2_mean',
 'burntCO2_std',
 'burntCO2_slope_ts',
 'burntCO2_pval_ts',
 'burntCO2_relchg']

In [62]:
sel_cols = ['GridNum10km',
 'LEVEL3_name',
 'LEVEL3_code',
 'ADM_ID',
 'ISO2',
 'protected_area',
 'env_zones',
 'LULUCF_CODE',
 'LULUCF_DESCRIPTION',
 'AreaHa',
 'year',
 'FireHa',
 'co2fireTn',
 'burntCO2_dev',
 'burntCO2_anom',  
 'burntCO2_mean',
 'burntCO2_std',
 'burntCO2_trend',
 'burntCO2_slope_ts',
 'burntCO2_pval_ts',
 'burntCO2_relchg'
 'burntCO2_10km_count']

In [63]:
emissions = fires_FTA_co2fire_dev_anom.filter(sel_cols)

In [64]:
emissions.tail()

,GridNum10km,LEVEL3_name,LEVEL3_code,ADM_ID,ISO2,protected_area,env_zones,LULUCF_CODE,LULUCF_DESCRIPTION,AreaHa,year,FireHa,co2fireTn,burntCO2_dev,burntCO2_anom,burntCO2_mean,burntCO2_std,burntCO2_trend,burntCO2_slope_ts,burntCO2_pval_ts
1048075,14218411923996672,Bouches-du-Rhône,FRL04,468.0,FR,None,MDS,FL,Forest land,470.0,2022,470,647.597,0.0,0.0,47.052,39.268,1.0,4.112,0.00390
1048076,14218411923996672,Bouches-du-Rhône,FRL04,468.0,FR,None,MDS,GL,Grassland,472.0,2022,472,650.353,0.0,0.0,47.253,39.435,1.0,4.129,0.00392
1048077,14218411923996672,Bouches-du-Rhône,FRL04,468.0,FR,None,MDS,CL,Cropland,61.0,2022,61,84.050,0.0,0.0,6.107,5.096,1.0,0.534,0.00051
1048078,14378347916165120,West,IE042,507.0,IE,None,ATC,WL,Wetlands,79.0,2022,0,0.000,0.0,0.0,NaN,NaN,0.0,0.000,0.00000
1048079,14378347916165120,West,IE042,507.0,IE,None,ATC,GL,Grassland,1.0,2022,0,0.000,0.0,0.0,NaN,NaN,0.0,0.000,0.00000


In [65]:
emissions.rename(columns= {'co2fireTn': 'burntCO2_Tn'}, inplace=True)
emissions.head()

,GridNum10km,LEVEL3_name,LEVEL3_code,ADM_ID,ISO2,protected_area,env_zones,LULUCF_CODE,LULUCF_DESCRIPTION,AreaHa,year,FireHa,burntCO2_Tn,burntCO2_dev,burntCO2_anom,burntCO2_mean,burntCO2_std,burntCO2_trend,burntCO2_slope_ts,burntCO2_pval_ts
0,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,None,ATN,FL,Forest land,37.0,2003,0,0.000,0.0,0.0,0.000,0.000,0.0,0.0,0.00000
1,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,None,ATN,GL,Grassland,26.0,2003,0,0.000,0.0,0.0,0.000,0.000,0.0,0.0,0.00000
2,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,Protected,ATN,FL,Forest land,1.0,2003,0,0.000,0.0,0.0,0.000,0.000,0.0,0.0,0.00000
3,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,Protected,ATN,WL,Wetlands,724.0,2003,722,843.147,0.0,0.0,0.768,2.872,0.0,0.0,0.37668
4,14434530383364096,"Lochaber, Skye & Lochalsh, Arran & Cumbrae and...",UKM63,1352.0,GB,Protected,ATN,GL,Grassland,81.0,2003,17,19.853,0.0,0.0,0.018,0.068,0.0,0.0,0.00887


In [66]:
emissions.query('GridNum10km == 	27400272145809408')

,GridNum10km,LEVEL3_name,LEVEL3_code,ADM_ID,ISO2,protected_area,env_zones,LULUCF_CODE,LULUCF_DESCRIPTION,AreaHa,year,FireHa,burntCO2_Tn,burntCO2_dev,burntCO2_anom,burntCO2_mean,burntCO2_std,burntCO2_trend,burntCO2_slope_ts,burntCO2_pval_ts
3261,27400272145809408,Adana,TR621,869.0,TR,Protected,MDS,WL,Wetlands,26.0,2003,0,0.000,0.000,0.000,NaN,NaN,-1.0,0.000,0.0
3262,27400272145809408,Adana,TR621,869.0,TR,Protected,MDS,CL,Cropland,1.0,2003,0,0.000,0.000,0.000,NaN,NaN,-1.0,0.000,0.0
55665,27400272145809408,Adana,TR621,869.0,TR,Protected,MDS,WL,Wetlands,26.0,2004,0,0.000,0.000,0.000,NaN,NaN,-1.0,0.000,0.0
55666,27400272145809408,Adana,TR621,869.0,TR,Protected,MDS,CL,Cropland,1.0,2004,0,0.000,0.000,0.000,NaN,NaN,-1.0,0.000,0.0
108069,27400272145809408,Adana,TR621,869.0,TR,Protected,MDS,WL,Wetlands,26.0,2005,0,0.000,0.000,0.000,NaN,NaN,-1.0,0.000,0.0
108070,27400272145809408,Adana,TR621,869.0,TR,Protected,MDS,CL,Cropland,1.0,2005,0,0.000,0.000,0.000,NaN,NaN,-1.0,0.000,0.0
160473,27400272145809408,Adana,TR621,869.0,TR,Protected,MDS,WL,Wetlands,26.0,2006,0,0.000,0.000,0.000,NaN,NaN,-1.0,0.000,0.0
160474,27400272145809408,Adana,TR621,869.0,TR,Protected,MDS,CL,Cropland,1.0,2006,0,0.000,0.000,0.000,NaN,NaN,-1.0,0.000,0.0
212877,27400272145809408,Adana,TR621,869.0,TR,Protected,MDS,WL,Wetlands,26.0,2007,0,0.000,0.000,0.000,NaN,NaN,-1.0,0.000,0.0
212878,27400272145809408,Adana,TR621,869.0,TR,Protected,MDS,CL,Cropland,1.0,2007,0,0.000,0.000,0.000,NaN,NaN,-1.0,0.000,0.0


In [45]:
emissions.query('GridNum10km == 	27400272145809408')

,GridNum10km,LEVEL3_name,LEVEL3_code,ADM_ID,ISO2,protected_area,env_zones,LULUCF_CODE,LULUCF_DESCRIPTION,AreaHa,year,FireHa,burntCO2_Tn,burntCO2_dev,burntCO2_anom,burntCO2_10km_mean,burntCO2_10km_std,burntCO2_10km_count
3261,27400272145809408,Adana,TR621,869.0,TR,Protected,MDS,WL,Wetlands,26.0,2003,0,0.000,0.000,0.000,1255.666626,663.350769,20
3262,27400272145809408,Adana,TR621,869.0,TR,Protected,MDS,CL,Cropland,1.0,2003,0,0.000,0.000,0.000,1255.666626,663.350769,20
55665,27400272145809408,Adana,TR621,869.0,TR,Protected,MDS,WL,Wetlands,26.0,2004,0,0.000,0.000,0.000,1255.666626,663.350769,20
55666,27400272145809408,Adana,TR621,869.0,TR,Protected,MDS,CL,Cropland,1.0,2004,0,0.000,0.000,0.000,1255.666626,663.350769,20
108069,27400272145809408,Adana,TR621,869.0,TR,Protected,MDS,WL,Wetlands,26.0,2005,0,0.000,0.000,0.000,1255.666626,663.350769,20
108070,27400272145809408,Adana,TR621,869.0,TR,Protected,MDS,CL,Cropland,1.0,2005,0,0.000,0.000,0.000,1255.666626,663.350769,20
160473,27400272145809408,Adana,TR621,869.0,TR,Protected,MDS,WL,Wetlands,26.0,2006,0,0.000,0.000,0.000,1255.666626,663.350769,20
160474,27400272145809408,Adana,TR621,869.0,TR,Protected,MDS,CL,Cropland,1.0,2006,0,0.000,0.000,0.000,1255.666626,663.350769,20
212877,27400272145809408,Adana,TR621,869.0,TR,Protected,MDS,WL,Wetlands,26.0,2007,0,0.000,0.000,0.000,1255.666626,663.350769,20
212878,27400272145809408,Adana,TR621,869.0,TR,Protected,MDS,CL,Cropland,1.0,2007,0,0.000,0.000,0.000,1255.666626,663.350769,20


In [67]:
emissions.describe()

,GridNum10km,ADM_ID,AreaHa,year,FireHa,burntCO2_Tn,burntCO2_dev,burntCO2_anom,burntCO2_mean,burntCO2_std,burntCO2_trend,burntCO2_slope_ts,burntCO2_pval_ts
count,1.048080e+06,967500.000000,967500.000000,1.048080e+06,1.048080e+06,1.048080e+06,1.048080e+06,1.048080e+06,163866.000000,163866.000000,976280.000000,1.048080e+06,1.048080e+06
mean,1.875279e+16,806.632062,186.354935,2.012500e+03,1.172532e+01,2.763709e+02,1.270344e+01,9.133101e-03,320.703714,286.300369,0.003892,-6.694725e-01,1.424356e-02
std,5.899324e+15,417.348798,471.260920,5.766284e+00,1.119533e+02,3.547859e+03,4.209165e+02,1.746764e-01,1455.958447,1087.777836,0.323967,1.768753e+01,8.004112e-02
min,4.637216e+15,0.000000,1.000000,2.003000e+03,0.000000e+00,0.000000e+00,-4.425713e+04,-2.082000e+00,0.000000,0.000000,-1.000000,-3.580250e+03,0.000000e+00
25%,1.411054e+16,467.000000,10.000000,2.007750e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
50%,1.870431e+16,742.000000,40.000000,2.012500e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.586500,3.692500,0.000000,0.000000e+00,0.000000e+00
75%,2.313506e+16,1067.000000,151.000000,2.017250e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,113.909750,119.032250,0.000000,0.000000e+00,0.000000e+00
max,3.214322e+16,1604.000000,9231.000000,2.022000e+03,9.112000e+03,5.207385e+05,4.178266e+04,6.722400e+01,149406.391000,41790.387000,1.000000,7.696430e+02,1.000000e+00


### 5.. Export tables

In [68]:
name_of_table = 'burntCO2_emissions'
emissions.to_sql(name_of_table, engine, if_exists='replace', index = False, schema='wild_fire')

-1